In [8]:
import os
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torch.optim import Adam

from torch_geometric.datasets import Planetoid
from torch_geometric.nn.models import InnerProductDecoder, VGAE
from torch_geometric.nn.conv import GCNConv
from torch_geometric.utils import negative_sampling, remove_self_loops, add_self_loops
import torch_geometric.transforms as T
from torch_geometric.utils import train_test_split_edges
from typing import Optional
from torch_scatter import scatter_add
from torch_sparse import coalesce
from torch_geometric.utils import add_self_loops, remove_self_loops, to_scipy_sparse_matrix
from torch_geometric.utils.num_nodes import maybe_num_nodes
from scipy.sparse.linalg import eigsh

import numpy as np
import scipy as sp
from numpy import linalg as LA
import seaborn as sns
from scipy.optimize import minimize
from scipy.sparse.linalg import eigs
np.random.seed(42)

In [9]:
from Mgnetic_conv import *

## Model

In [10]:
## VAGE MAX_LOGSTD = 10
MAX_LOGSTD = 10

class MagConvEncoder(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, k , q ,trainable_q):
        super(MagConvEncoder, self).__init__()
        self.magconv_shared = MagNetConv(in_channels, hidden_channels,k,q,trainable_q)
        self.magconv_mu = MagNetConv(hidden_channels, out_channels,k,q,trainable_q)
        self.magconv_logvar = MagNetConv(hidden_channels, out_channels,k,q,trainable_q)

    def forward(self, x, edge_index):
        ## x_real & x_img as input
        x_real = x.real.float()
        x_imag_1 = x * 1j
        x_imag = x_imag_1.float()
        # print(x_real,x_imag_1,x_imag)
        # all_zeros = torch.all(x_real == 0)
        # print("Are all elements zeros?", all_zeros)
        # x_imag = x.clone
        ### should be complex relu

###ReLU
## 3RELU -- mu and log !use x -- medium --  80+
## 3RELU -- mu and log use x -- stuck
## 1Act == 0Act-- since mu and log do not include x -- 94.8 optimal
## 1RELU（first layter） -- mu and log indeed include x -- 93
## 0 act -- mu and log include x -- 91 
        x = self.magconv_shared(x_real,x_imag,edge_index)[0]
        mu = self.magconv_mu(x_real,x_imag,edge_index)[0]
        logvar = self.magconv_logvar(x_real,x_imag,edge_index)[0]
        return mu,logvar
        # x_2 = F.relu(self.magconv_shared(x_real,x_imag,edge_index)[0])
        # x_real_2 = x_2.real.float()
        # x_imag_2 = x_2.real * 1j 
        # x_imag_2 = x_imag_2.float()
        # mu = F.relu(self.magconv_mu(x_real_2,x_imag_2,edge_index)[0])
        # logvar = F.relu(self.magconv_logvar(x_real_2,x_imag_2,edge_index)[0])
        # return mu,logvar


class SyncRankDecoder(nn.Module):
    def __init__(self):
        super(SyncRankDecoder, self).__init__()


    def forward(self,z,g = 0.5):
        ##  regard latent representation as the input comparison matrix to the syncrank decoder
        C = z
        n = C.shape[0]
        Theta = 2 * np.pi * g * C / (n-1)
        # H = np.exp(1j * Theta)
        # H = torch.exp(1j * Theta)
        # d = torch.sum(torch.abs(H), dim=1).detach() + 1e-10
        # v, psi = eigs(1j * torch.diag(d**(-1)) @ H, 1, which='LR')
        H = torch.exp(1j * Theta)
        d = torch.sum(torch.abs(H), dim=1).detach() + 1e-10

        # Convert d to a diagonal matrix
        D_inv = torch.diag(1.0 / d)
        # Compute eigenvalues and eigenvectors using torch.linalg.eig
        H = torch.tensor(H, dtype=torch.complex64)  # Convert H to a complex tensor
        A = 1j * D_inv @ H
        eigenvalues, eigenvectors = torch.linalg.eig(A)
        # Get the largest real part eigenvalue and its corresponding eigenvector
        largest_eigenvalue_idx = torch.argmax(eigenvalues.real)
        psi = eigenvectors[:, largest_eigenvalue_idx].real


        # d = np.sum(np.abs(H),axis = 1) + 1e-10
        # v,psi = eigs(1j * np.diag(d**(-1)) @ H,1,which = 'LR')
        # r_hat =  psi / np.abs(psi)
        # r_hat = r_hat.reshape(n)
        psi = psi.reshape(n)
        angles_radians = np.angle(psi)
        angles_modulo_2pi = np.mod(angles_radians, 2 * np.pi)
        angles_degrees = np.degrees(angles_modulo_2pi)
        print('angle_degrees is ',angles_degrees)
        sorted_indices = np.argsort(angles_degrees)
        print('label of degree from smallest to largest',sorted_indices)
        return sorted_indices ## decoder here return the label --  project the latent space to a new angle space 
        



class DeepVGAE(VGAE):
    def __init__(self,in_channels:int,hidden_channels:int, out_channels:int, K:int, q:float, trainable_q:bool):
        super(DeepVGAE, self).__init__(encoder= MagConvEncoder(in_channels,
                                                        hidden_channels,
                                                        out_channels,
                                                        K,
                                                        q,
                                                        trainable_q
                                                        ),
                                        decoder = SyncRankDecoder())
        self.out_channels = out_channels

    ## embedding z
    def encode(self,x,edge_index):
        # print(self.encoder)
        self.__mu__, self.__logstd__ = self.encoder(x,edge_index)
        # gaussian_noise = torch.randn(x.size(0), self.out_channels)
        # print(self.training)
        # z = self.__mu__ + gaussian_noise * torch.exp(self.__logstd__ )
        z = self.reparametrize(self.__mu__, self.__logstd__)
        return z

    def forward(self,x,edge_index):
        x_real = x.real.float()
        z = self.encode(x_real, edge_index)
        ## forward_all 
        output = self.decoder.forward(z)
        return output

    def loss(self, x, pos_edge_index):
        z = self.encode(x, pos_edge_index)
        # Original Loss function
        # pos_loss = -torch.log(
        #     self.decoder(z, pos_edge_index) + 1e-15).mean()
        # neg_edge_index = negative_sampling(all_edge_index, z.size(0), pos_edge_index.size(1))
        # neg_loss = -torch.log(1 - self.decoder(z, neg_edge_index) + 1e-15).mean()
        # kl_loss = 1 / x.size(0) * self.kl_loss() 

        ##new losss funciton
        display(z)
        print(z.shape)
        n = z.shape[0]
        s_projection = self.decoder(z)
        obj = 9999999999999
        best_shift = 0
        for shift in range(n):
            obj1 = self.upset(s_projection,z,shift)
            print(obj1)
            if obj1 < obj:
                obj = obj1
                best_shift = shift
                # print(circular_shift(s,best_shift))
        repre = self.circular_shift(s_projection,best_shift)
        obj_loss = obj
        return obj_loss
        
    def circular_shift(self, x, shift):
            return np.concatenate((x[-shift:], x[:-shift]))
    
    def outer_product(self, x, y):
        return np.outer(x, y)
    
    def hadamard_product(self, x, y):
        return x * y
    
    def upset(self, s, C, shift):
        sigma_s = self.circular_shift(s, shift)
        n = len(s)
        sigma_s = sigma_s.reshape(n, 1)
        sigma_outer_ones_T = self.outer_product(sigma_s, np.ones(len(sigma_s)))
        ones_outer_sigma_T = self.outer_product(np.ones(len(sigma_s)), sigma_s.T)
        term1 = sigma_outer_ones_T - ones_outer_sigma_T
        result = self.hadamard_product(term1, [C!=0])
        return 0.5 * (np.sum( np.abs(np.sign(result) - np.sign(C))))


    def single_test(self, x, train_pos_edge_index, test_pos_edge_index, test_neg_edge_index):
        with torch.no_grad():
            z = self.encode(x, train_pos_edge_index)
        roc_auc_score, average_precision_score = self.test(z, test_pos_edge_index, test_neg_edge_index)
        return roc_auc_score, average_precision_score

In [11]:
from torch.optim import Adam
torch.manual_seed(3407)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
os.makedirs("datasets", exist_ok=True)
dataset = Planetoid("datasets",'Cora', transform=T.NormalizeFeatures())
data = dataset[0].to(device)
all_edge_index = data.edge_index
data = train_test_split_edges(data, 0.05, 0.1)
data

/Users/leoenoch/opt/anaconda3/lib/python3.9/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


Data(x=[2708, 1433], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], val_pos_edge_index=[2, 263], test_pos_edge_index=[2, 527], train_pos_edge_index=[2, 8976], train_neg_adj_mask=[2708, 2708], val_neg_edge_index=[2, 263], test_neg_edge_index=[2, 527])

In [13]:
model = DeepVGAE(1433,1433,2708,2,0.25,True)
optimizer = Adam(model.parameters(), lr= 0.01)

In [ ]:
model.__repr__

<bound method Module.__repr__ of DeepVGAE(
  (encoder): MagConvEncoder(
    (magconv_shared): MagNetConv(1433, 1433, K=2, normalization=sym)
    (magconv_mu): MagNetConv(1433, 2708, K=2, normalization=sym)
    (magconv_logvar): MagNetConv(1433, 2708, K=2, normalization=sym)
  )
  (decoder): SyncRankDecoder()
)>

In [ ]:
for epoch in range(500):
    model.train()
    optimizer.zero_grad()
    loss = model.loss(data.x, data.train_pos_edge_index) ##self.train = True
    loss.backward()
    optimizer.step()
    if epoch % 2 == 0:
      model.eval()
      roc_auc, ap = model.single_test(data.x,
                                        data.train_pos_edge_index,
                                        data.test_pos_edge_index,
                                        data.test_neg_edge_index) ## self.train = False
      print("Epoch {} - Loss: {} ROC_AUC: {} Precision: {}".format(epoch, loss.cpu().item(), roc_auc, ap))